# Worksheet 05

Name: Yuchen Cao  
UID: U51424608

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

First Step:  
Assign each point in the dataset to its closest center:  
centroid 0: 0, 0.5,  
centroid 2: 1.5, 2, 6, 6.5, 7  
Compute the new centers as the means of each cluster: (0 + 0.5) / 2 = 0.25, (1.5 + 2 + 6 + 6.5 + 7) / 5 = 4.6  
Then the new centroids are [0.25, 4.6]

Second Step:  
Assign each point in the dataset to its closest center:  
centroid 0.25: 0, 0.5, 1.5, 2  
centroid 4.6: 6, 6.5, 7  
Compute the new centers as the means of each cluster: (0 + 0.5 + 1.5 + 2) / 4 = 1, (6 + 6.5 + 7) / 3 = 6.5  
Then the new centroids are [1, 6.5]

Third Step:  
Assign each point in the dataset to its closest center:  
centroid 1: 0, 0.5, 1.5, 2  
centroid 6.5: 6, 6.5, 7  
Compute the new centers as the means of each cluster: (0 + 0.5 + 1.5 + 2) / 4 = 1, (6 + 6.5 + 7) / 3 = 6.5  
Then the new centroids are still [1, 6.5], so it's convergence.

b) Describe in plain english what the cost function for k means is.

The mean of distance between each point to its own centroid.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Since we select the initial centroids randomly, so this could lead to the algorithm converging to different local optima rather than the global optimum.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, since it's a greedy algorithm and it always move to decrease the cost. Thus, the cost will always converge to a value, so does the algorithm.

e) Follow along in class the implementation of Kmeans

In [12]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def is_unassigned(self, i):
        return self.assignment[i] == -1

    def unassigned_all(self, i):
        self.assignment = [-1 for _ in range(len(data))]

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size = self.k, replace = False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, enters):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if new_dist < temp_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist

    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[np.array([i for i in range(len(self.data)) if self.assignment[i] == j])]
            centers.append(np.mean(cluster_j, axis = 0))
        return np.array(centers)

    def lloyds(self):
        centers = self.initialize()
        #self.snaps.append(self.snap(centers))
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while not self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.unassigned_all()
            self.assign(centers)
            self.snap(centers)
            new_centers = self.calculate_new_centers()

        return
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

IndexError: arrays used as indices must be of integer (or boolean) type